In [1]:
from d2l import torch as d2l
import torch
import torchvision
from torch import nn

In [2]:
class G_block(nn.Module):
        def __init__(self, channels, nc=3, kernel_size=4, strides=2,
                 padding=1, alpha=0.2, **kwargs):
            super(G_block, self).__init__(**kwargs)
            self.conv2d = nn.ConvTranspose2d(
                nc, channels, kernel_size, strides, padding, bias=False)
            self.batch_norm = nn.BatchNorm2d(channels)
            self.activation = nn.ReLU(alpha)

        def forward(self, X):
            return self.activation(self.batch_norm(self.conv2d(X)))

In [3]:
x = torch.zeros((2, 3, 16, 16))
g_blk = G_block(20)
g_blk(x).shape

torch.Size([2, 20, 32, 32])

In [4]:
x = torch.zeros((2, 3, 1, 1))
g_blk = G_block(20, strides=1, padding=0)
g_blk(x).shape

torch.Size([2, 20, 4, 4])

In [5]:
def Conv2DTranspose(channels, kernel_size, strides, padding, use_bias, nc=3):
    return nn.ConvTranspose2d(nc, channels, kernel_size=kernel_size,stride=strides, padding=padding, bias=use_bias)

In [6]:
n_G = 64
net_G = nn.Sequential(
    G_block(n_G*8, strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(n_G*4, n_G*8),  # Output: (64 * 4, 8, 8)
    G_block(n_G*2, n_G*4),  # Output: (64 * 2, 16, 16)
    G_block(n_G, n_G*2),    # Output: (64, 32, 32)
    Conv2DTranspose(
              3, nc=n_G, kernel_size=4, strides=2, padding=1, use_bias=False),
    nn.Tanh()               # Output: (3, 64, 64)
)

In [7]:
x = torch.zeros((1, 3, 1, 1))
net_G(x).shape

torch.Size([1, 3, 64, 64])